# NLP Project using Spark

For this code along we will build a spam filter! We'll use the various NLP tools we learned about as well as a new classifier, Naive Bayes.

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [28]:
from pyspark.sql import SparkSession

In [29]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [30]:
data = spark.read.csv("../datasets/SMSSpamCollection",inferSchema=True,sep='\t')

In [31]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [32]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



## Clean and Prepare the Data

** Create a new length feature: **

In [33]:
from pyspark.sql.functions import length

In [34]:
data = data.withColumn('length',length(data['text']))

In [35]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [36]:
# Pretty Clear Difference
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

In [37]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [38]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [39]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

### The Model

We'll use Naive Bayes, but feel free to play around with this choice!

In [40]:
from pyspark.ml.classification import NaiveBayes

In [41]:
# Use defaults
nb = NaiveBayes()

### Pipeline

In [42]:
from pyspark.ml import Pipeline

In [43]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [44]:
cleaner = data_prep_pipe.fit(data)

In [45]:
clean_data = cleaner.transform(data)

### Training and Evaluation!

In [46]:
clean_data = clean_data.select(['label','features'])

In [47]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [48]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [49]:
spam_predictor = nb.fit(training)

In [50]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [51]:
test_results = spam_predictor.transform(testing)

In [52]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-807.85430102661...|[1.0,4.4092579857...|       0.0|
|  0.0|(13424,[0,1,2,41,...|[-1077.5377991882...|[1.0,1.1632130699...|       0.0|
|  0.0|(13424,[0,1,3,9,1...|[-570.94294546402...|[1.0,8.1052712228...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-801.27699612116...|[1.0,9.4072746096...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-882.11701105115...|[1.0,5.3456886827...|       0.0|
|  0.0|(13424,[0,1,14,31...|[-217.03030294557...|[1.0,5.0002531356...|       0.0|
|  0.0|(13424,[0,1,17,19...|[-804.90457257807...|[1.0,3.4749002636...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-969.60401679996...|[1.0,7.4881874560...|       0.0|
|  0.0|(13424,[0,1,23,63...|[-1317.5715795344...|[1.0,1.7454057728...|       0.0|
|  0.0|(13424,[0

In [53]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [54]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.915258928824168


Not bad considering we're using straight math on text data! Try switching out the classification models! Or even try to come up with other engineered features!

## Great Job!